In [1]:
import numpy as np
import pandas as pd
from skimage.color import rgb2gray
from scipy import ndimage
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import os

path = os.listdir(r'C:\Users\Administrator\Downloads\Training')
classes = {'no_tumor':0, 'pituitary_tumor':1}

FileNotFoundError: ignored

# Edge base segmentation

In [ ]:
image = cv2.imread('Sample.png')
image.shape
plt.imshow(image)


In [ ]:
gray = rgb2gray(image)

sobel_horizontal = np.array([np.array([1, 2, 1]), np.array([0, 0, 0]), np.array([-1, -2, -1])])
sobel_vertical = np.array([np.array([-1, 0, 1]), np.array([-2, 0, 2]), np.array([-1, 0, 1])])

out_h = ndimage.convolve(gray, sobel_horizontal, mode='reflect')
out_v = ndimage.convolve(gray, sobel_vertical, mode='reflect')

In [ ]:
plt.imshow(out_h, cmap='gray')

In [ ]:
plt.imshow(out_v, cmap='gray')

# Region based segmentation

In [ ]:
gray = rgb2gray(image)
plt.imshow(gray, cmap='gray')

In [ ]:
gray.shape

In [ ]:
gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
for i in range(gray_r.shape[0]):
    if gray_r[i] > gray_r.mean():
        gray_r[i] = 1
    else:
        gray_r[i] = 0
gray = gray_r.reshape(gray.shape[0],gray.shape[1])
plt.imshow(gray, cmap='gray')

# Clustering based segmentation

In [ ]:
original_image = cv2.imread('p.jpg')
plt.imshow(original_image)

In [ ]:
img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
vectorized = img.reshape((-1,3))
vectorized = np.float32(vectorized)


In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 3
attempts=10
ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)

In [ ]:
center = np.uint8(center)

res = center[label.flatten()]
result_image = res.reshape((img.shape))

In [ ]:
figure_size = 15
plt.figure(figsize=(figure_size,figure_size))
plt.subplot(1,2,1),plt.imshow(img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(result_image)
plt.title('Segmented Image when K = %i' % K), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
import cv2
X = []
Y = []
for cls in classes:
    pth = 'C:/Users/Administrator/Downloads/Training/' + cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j, 0)
        img = cv2.resize(img, (200,200))
        X.append(img)
        Y.append(classes[cls])


In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
np.unique(Y)

In [ ]:

pd.Series(Y).value_counts()

In [ ]:
X.shape

# Visualisng The Data

In [ ]:
plt.imshow(X[0], cmap='gray')

In [ ]:
X_updated = X.reshape(len(X), -1)
X_updated.shape

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_updated, Y, random_state=10,
                                               test_size=.20)

In [ ]:
xtrain.shape, xtest.shape

In [ ]:
print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())
xtrain = xtrain/255
xtest = xtest/255
print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
print(xtrain.shape, xtest.shape)

pca = PCA(.98)
# pca_train = pca.fit_transform(xtrain)
# pca_test = pca.transform(xtest)
pca_train = xtrain
pca_test = xtest

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
import warnings
warnings.filterwarnings('ignore')

lg = LogisticRegression(C=0.1)
lg.fit(pca_train, ytrain)

In [ ]:
sv = SVC()
sv.fit(pca_train, ytrain)

In [ ]:
print("Training Score:", lg.score(pca_train, ytrain))
print("Testing Score:", lg.score(pca_test, ytest))

In [ ]:
print("Training Score:", sv.score(pca_train, ytrain))
print("Testing Score:", sv.score(pca_test, ytest))

In [ ]:
pred = sv.predict(pca_test)
np.where(ytest!=pred)

In [ ]:
pred[36]

In [ ]:
ytest[36]

In [ ]:
dec = {0:'No Tumor', 1:'Positive Tumor'}

In [ ]:
plt.figure(figsize=(12,8))
p = os.listdir(r'C:/Users/Administrator/Downloads/Testing')
c=1
for i in os.listdir(r'C:/Users/Administrator/Downloads/Testing/no_tumor/')[:9]:
    plt.subplot(3,3,c)
    
    img = cv2.imread(r'C:/Users/Administrator/Downloads/Testing/no_tumor/'+i,0)
    img1 = cv2.resize(img, (200,200))
    img1 = img1.reshape(1,-1)/255
    p = sv.predict(img1)
    plt.title(dec[p[0]])
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    c+=1

In [ ]:
plt.figure(figsize=(12,8))
p = os.listdir(r'C:/Users/Administrator/Downloads/Testing')
c=1
for i in os.listdir(r'C:/Users/Administrator/Downloads/Testing/pituitary_tumor/')[:16]:
    plt.subplot(4,4,c)
    
    img = cv2.imread(r'C:/Users/Administrator/Downloads/Testing/pituitary_tumor/'+i,0)
    img1 = cv2.resize(img, (200,200))
    img1 = img1.reshape(1,-1)/255
    p = sv.predict(img1)
    plt.title(dec[p[0]])
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    c+=1